<a href="https://colab.research.google.com/github/AlsouAR/HW_p_1/blob/main/%D0%90%D0%B1%D1%81%D0%B0%D1%82%D1%82%D0%B0%D1%80%D0%BE%D0%B2%D0%B0_%D0%90%D0%BB%D1%81%D1%83(%D0%91%D0%94).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Работа с базами данных SQLite в Python

Решение задач для самостоятельной работы (суть заданий написана в комментариях)

In [1]:
import sqlite3
import os


def show(rows):
    for row in rows:
        print(row)
    print('\n')


# 1. Создание базы данных и таблиц
# Подключение к базе данных
connection = sqlite3.connect('library.db')
cursor = connection.cursor()

# SQL-запрос для создания таблицы
create_table_query = '''
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    year_published INTEGER,
    genre TEXT
);
'''
# Выполнение запроса
cursor.execute(create_table_query)

# 14. Триггер для логирования
# записывает изменения в таблицу logs при добавлении новой книги
# Создание таблицы для логирования изменений
cursor.execute('''
CREATE TABLE IF NOT EXISTS book_logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    book_id INTEGER,
    action TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
);
''')

# Создание тригера
create_trigger_query = '''
CREATE TRIGGER IF NOT EXISTS log_book_changes
AFTER INSERT ON books
FOR EACH ROW
BEGIN
    INSERT INTO book_logs (book_id, action)
    VALUES (NEW.id, 'INSERT');
END;
'''
cursor.execute(create_trigger_query)
connection.commit()

# 2. Вставка данных
# SQL-запрос для вставки данных
insert_query = '''
INSERT INTO books (title, author, year_published, genre)
VALUES (?, ?, ?, ?);
'''
user_data = [('The Great Gatsby', 'F.Scott Fitzgerald', 1925, 'Fiction'),
             ('To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic'),
             ('1984', 'George Orwell', 1949, 'Dystopian'),
             ]
cursor.executemany(insert_query, user_data)
# Сохранение изменений
connection.commit()

# 3. Выборка всех данных
# сделано в функции чтобы потом использовать
select_query = '''
        SELECT * FROM books;
        '''
def show_all(select_query):
    cursor.execute(select_query)
    rows = cursor.fetchall()
    show(rows)
print("Выборка всех данных")
show_all(select_query)

# 4. Фильтрация данных
query = '''
SELECT * FROM books WHERE year_published > ?;
'''
age_limit = 1950
cursor.execute(query, (age_limit,))
rows = cursor.fetchall()
print('Книги, опубликованные после 1950 года.')
show(rows)

# 5. Использование оператора LIKE
query = '''
SELECT * FROM books WHERE title LIKE ?;
'''
search_pattern = 'T%'
cursor.execute(query, (search_pattern,))
rows = cursor.fetchall()
print('Книги, название которых начинается с буквы "T".')
show(rows)

# 6. Сортировка данных
query = '''
SELECT * FROM books ORDER BY year_published ASC
'''
cursor.execute(query)
sorted_books = cursor.fetchall()
print("Книги, отсортированные по году публикации:")
show(sorted_books)

# 7. Ограничение количества строк
# SQL-запрос для выборки данных
query = '''
SELECT * FROM books
ORDER BY title ASC
LIMIT 2;
'''
cursor.execute(query)
rows = cursor.fetchall()
print('Первые две книги')
show(rows)

# 8. Агрегатные функции
# Общее количество книг в таблице books
query = '''
SELECT COUNT(*) FROM books;
'''
cursor.execute(query)
result = cursor.fetchone()
print(f"Общее количество книг: {result[0]}")

# 9. Группировка данных
# количество книг для каждого жанра.
query = '''
SELECT genre, COUNT(*) as book_count
FROM books
GROUP BY genre
ORDER BY book_count DESC
'''
cursor.execute(query)
genre_stats = cursor.fetchall()
print("Количество книг по жанрам:")
for genre, count in genre_stats:
    print(f"{genre}: {count}")
print()

# 10. Обновление данных "1984" на 1948.
# SQL-запрос для обновления данных
update_query = '''
UPDATE books
SET year_published  = 1948
WHERE title  = '1984';
'''
cursor.execute(update_query)
connection.commit()
print('Обновление данных "1984" на 1948:')
show_all(select_query)

# 11. Удаление данных Удалите книгу с названием "The Great Gatsby"
delete_query = '''
DELETE FROM books
WHERE title = ?;
'''
book_title = "The Great Gatsby"
cursor.execute(delete_query, (book_title,))
connection.commit()
print('Удалена книга с названием "The Great Gatsby":')
show_all(select_query)

# 12. Создание индекса, индекс для столбца author в таблице books
create_index_query = '''
CREATE INDEX IF NOT EXISTS idx_books_author
ON books (author)
'''
cursor.execute(create_index_query)
connection.commit()

# 13. Проверка существования индекса
cursor.execute('''
SELECT name FROM sqlite_master
WHERE type = 'index' AND name = 'idx_books_author'
''')
index_exists = cursor.fetchone()

if index_exists:
    print("Индекс idx_books_author существует")
else:
    print("Индекс idx_books_author не существует")

# 14 проверка тригера вывод
select_logs_query = '''
SELECT * FROM book_logs;
'''
print("Логи изменений:")
show_all(select_logs_query)

# 15. Транзакции
# Добавляет новую книгу.
# Обновляет год публикации другой книги.
# Отменяет изменения, если возникает ошибка
try:
    # Начало транзакции
    cursor.execute('BEGIN TRANSACTION')

    # Вставка данных
    cursor.execute('''
    INSERT INTO books (title, author, year_published, genre)
    VALUES (?, ?, ?, ?);
    ''', ('The Hobbit', 'J.R.R. Tolkien', 1937, 'Fantasy'))

    # Обновление данных
    cursor.execute('''
    UPDATE books
    SET year_published = year_published + 1
    WHERE title = ?;
    ''', ('To Kill a Mockingbird',))

    # Фиксация изменений
    connection.commit()
    print("Транзакция успешно завершена.")
except Exception as e:
    # Откат изменений в случае ошибки
    connection.rollback()
    print(f"Ошибка: {e}. Изменения отменены.")

print('Данные после транзакции:')
show_all(select_query)


# 16. Пользовательская функция
# которая вычисляет возраст книги
def calculate_age(year_published):
    from datetime import datetime
    current_year = datetime.now().year
    return current_year - year_published


# Регистрация пользовательской функции
connection.create_function("calculate_age", 1, calculate_age)
# Выполнение запроса с использованием пользовательской функции
cursor.execute('''
SELECT title, calculate_age(year_published) AS age FROM books;
''')
rows = cursor.fetchall()
print('Возраст книг')
show(rows)

# 17. Представления
# книги, опубликованные после 1950 года
cursor.execute('''
CREATE VIEW IF NOT EXISTS modern_books AS
SELECT * FROM books WHERE year_published > 1950;
''')

# Выполнение запроса с использованием представления
cursor.execute('SELECT * FROM modern_books;')

# Получение результатов
print('Книги, опубликованные после 1950 года')
rows = cursor.fetchall()
show(rows)

# 19. Композитный индекс
# для столбцов author и year_published в таблице books
try:
    cursor.execute('''
    CREATE INDEX IF NOT EXISTS idx_books_author_year_published
    ON books (author, year_published);
    ''')
    connection.commit()
    print("Композитный индекс idx_books_author_year_published успешно создан")
except Exception as e:
    print(f"Ошибка: {e}.")
# проверка
cursor.execute('''
    SELECT name FROM sqlite_master
    WHERE type = 'index' AND name = 'idx_books_author_year_published'
    ''')
if cursor.fetchone():
    print("Индекс существует в базе данных")
else:
    print("Индекс не был создан")
print()

# 18. Удаление таблицы
try:
    cursor.execute('''
    SELECT name FROM sqlite_master WHERE type='table' AND name='books';
    ''')
    result = cursor.fetchone()
    if result:
        print("Таблица 'books' существует. Удаляем...")
        cursor.execute('DROP TABLE books')
    else:
        print("Таблица 'books' не существует.")
    connection.commit()
    print('Таблица успешно удалена')
except Exception as e:
    print(f"Ошибка: {e}.")

connection.close()

# 20. Удаление базы данных
# Проверка существования файла
if os.path.exists('library.db'):
    print(f"Удаляем базу данных '{'library.db'}'...")
    os.remove('library.db')
else:
    print(f"База данных '{'library.db'}' не существует.")


Выборка всех данных
(1, 'The Great Gatsby', 'F.Scott Fitzgerald', 1925, 'Fiction')
(2, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')
(3, '1984', 'George Orwell', 1949, 'Dystopian')


Книги, опубликованные после 1950 года.
(2, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')


Книги, название которых начинается с буквы "T".
(1, 'The Great Gatsby', 'F.Scott Fitzgerald', 1925, 'Fiction')
(2, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')


Книги, отсортированные по году публикации:
(1, 'The Great Gatsby', 'F.Scott Fitzgerald', 1925, 'Fiction')
(3, '1984', 'George Orwell', 1949, 'Dystopian')
(2, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')


Первые две книги
(3, '1984', 'George Orwell', 1949, 'Dystopian')
(1, 'The Great Gatsby', 'F.Scott Fitzgerald', 1925, 'Fiction')


Общее количество книг: 3
Количество книг по жанрам:
Fiction: 1
Dystopian: 1
Classic: 1

Обновление данных "1984" на 1948:
(1, 'The Great Gatsby', 'F.Scott Fitzgerald', 1925, 'Fiction')
(2